In [1]:
import utils

# Configure logging for the pipeline
logger = utils.setup_logger(level='INFO')

In [2]:
# print(utils.get_available_cities())
cities = ['Aachen', 'Frankfurt', 'CapeTown', 'Hamburg', 'Johannesburg', 'London', 'Montreal', 'Paris', 'Seoul', 'Singapore', 'Sydney']

In [3]:
from data_acquisition import DataHandler

datahandler = DataHandler(logger)

for city in cities: 
    datahandler.get_buildings(city)
    datahandler.get_satellite_image(city)
    datahandler.get_building_mask(city)

# Plot the expected results for the first city 
datahandler.plot(city[0])

2024-06-24 11:07:56,575 - root - INFO - __init__ - Data directory already exists
2024-06-24 11:07:56,576 - root - INFO - create_directory - Aachen: Directory available
2024-06-24 11:07:56,577 - root - INFO - get_buildings - Aachen: Using local building data


In [ ]:
import data_preparation

for city in cities:
    data_preparation.create_tensor(city)